# Stokes equations using Taylor-Hood elements

This demo is implemented in `demo_stokes.py`. It shows how
to solve the Stokes problem using Taylor-Hood elements using different
linear solvers.

## Equation and problem definition

### Strong formulation

$$
\begin{aligned}
  - \nabla \cdot (\nabla u - p I) &= f \quad {\rm in} \ \Omega,\\
  \nabla \cdot u &= 0 \quad {\rm in} \ \Omega.
\end{aligned}
$$

with conditions on the boundary $\partial \Omega = \Gamma_{D} \cup
\Gamma_{N}$ of the form:

$$
\begin{aligned}
  u &= u_0 \quad {\rm on} \ \Gamma_{D},\\
  \nabla u \cdot n - p n &= g \,   \quad\;\; {\rm on} \ \Gamma_{N}.
\end{aligned}
$$

### Weak formulation

The weak formulation reads: find $(u, p) \in V \times Q$ such that

$$
a((u, p), (v, q)) = L((v, q)) \quad \forall  (v, q) \in V \times Q
$$

where

$$
\begin{aligned}
  a((u, p), (v, q)) &:= \int_{\Omega} \nabla u \cdot \nabla v -
           \nabla \cdot v \ p - \nabla \cdot u \ q \, {\rm d} x,\\
  L((v, q)) &:= \int_{\Omega} f \cdot v \, {\rm d} x + \int_{\partial
           \Omega_N} g \cdot v \, {\rm d} s.
\end{aligned}
$$

### Domain and boundary conditions

We consider the lid-driven cavity problem with the following
domain and boundary conditions:

- $\Omega := [0,L]\times[0,1]$ 
- $\Gamma_D := \partial \Omega \setminus \{x_1=L, x_2\in (0,1)\}$
- $u_0 := (x_2(1-x_2), 0)^\top$ at $x_1 = 0$ (inflow of the channel) and $u_0 = (0, 0)^\top$ for $x_2 \in\{0, 1\}$ (top and bottom) 
- $f := (0, 0)^\top$
- $g :=(0, 0)^\top$



The required modules are first imported:

In [1]:
from mpi4py import MPI
from petsc4py import PETSc
import numpy as np
import ufl
from basix.ufl import element, mixed_element
from dolfinx import default_real_type, fem, la
from dolfinx.fem import (
    Constant,
    Function,
    dirichletbc,
    extract_function_spaces,
    form,
    functionspace,
    locate_dofs_topological,
    petsc
)

from dolfinx.mesh import CellType, create_rectangle, locate_entities_boundary
from dolfinx.plot import vtk_mesh
from ufl import div, dx, grad, inner
import pyvista
pyvista.set_jupyter_backend('html')


We create a {py:class}`Mesh <dolfinx.mesh.Mesh>`, define functions for
locating geometrically subsets of the boundary, and define a function
for the  velocity on the lid:

In [2]:
L = 4
# Create mesh
msh = create_rectangle(
    MPI.COMM_WORLD, [np.array([0, 0]), np.array([L, 1])], [16*L, 16], CellType.triangle
)

# Function to mark x = 0, x = 1 and y = 0
def noslip_boundary(x):
    return np.isclose(x[1], 0.0) | np.isclose(x[1], 1.0) 

# Function to mark the inflow
def inflow(x):
    return np.isclose(x[0], 0.0)

# Inflow velocity
# def inflow_velocity_expression(x):
    # return np.stack(( x[1]*(1-x[1]), np.zeros(x.shape[1])))
def inflow_velocity_expression(x):
    return np.stack(( 1./2*np.ones(x.shape[1]), np.zeros(x.shape[1])))

Two {py:class}`function spaces <dolfinx.fem.FunctionSpace>` are
defined using different finite elements. `P2` corresponds to a
continuous piecewise quadratic basis (vector) and `P1` to a continuous
piecewise linear basis (scalar).

In [3]:
P2 = element("Lagrange", msh.basix_cell(), 2, shape=(msh.geometry.dim,), dtype=default_real_type)
P1 = element("Lagrange", msh.basix_cell(), 1, dtype=default_real_type)
# Create the Taylot-Hood function space
TH = mixed_element([P2, P1])
W = functionspace(msh, TH)

# No slip boundary condition
W0 = W.sub(0)
Q, _ = W0.collapse()


Boundary conditions

In [4]:
noslip = Function(Q)
facets = locate_entities_boundary(msh, 1, noslip_boundary)
dofs = locate_dofs_topological((W0, Q), 1, facets)
bc0 = dirichletbc(noslip, dofs, W0)

# Driving velocity condition u = (1, 0) on top boundary (y = 1)
inflow_velocity = Function(Q)
inflow_velocity.interpolate(inflow_velocity_expression)
facets = locate_entities_boundary(msh, 1, inflow)
dofs = locate_dofs_topological((W0, Q), 1, facets)
bc1 = dirichletbc(inflow_velocity, dofs, W0)

# Collect Dirichlet boundary conditions
bcs = [bc0, bc1]



Function for plotting the flow and pressure

In [5]:
def plot(u, p):
    V = u.function_space
    msh = V.mesh
    topology, cell_types, geometry = vtk_mesh(V)
    values = np.zeros((geometry.shape[0], 3), dtype=np.float64)
    values[:, :len(u)] = u.x.array.real.reshape((geometry.shape[0], len(u)))

    # Create a point cloud of glyphs
    function_grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
    function_grid["u"] = values
    glyphs = function_grid.glyph(orient="u", factor=0.4)



    # Create a pyvista-grid for the mesh
    msh.topology.create_connectivity(msh.topology.dim, msh.topology.dim)
    grid = pyvista.UnstructuredGrid(*vtk_mesh(msh, msh.topology.dim))

    grid.point_data["p"] = p.x.array.real
    grid.set_active_scalars("p")
    # Create plotter
    plotter = pyvista.Plotter()
    plotter.add_mesh(grid, color='black', style="wireframe")
    pressure = plotter.add_mesh(grid, cmap="Greys")
    flow = plotter.add_mesh(glyphs, name="u")
    plotter.view_xy()
    
    def toggle_vis_p(flag):
        pressure.SetVisibility(flag)

    def toggle_vis_u(flag):
        flow.SetVisibility(flag)
    plotter.add_checkbox_button_widget(toggle_vis_p, position=(10, 10), value=True)
    plotter.add_checkbox_button_widget(toggle_vis_u, position=(10, 80), value=True)
    plotter.add_text('Show/hide pressure', position=(70, 12), color='black', shadow=True, font_size=12)
    plotter.add_text('Show/hide flow', position=(70, 82), color='black', shadow=True, font_size=12)

    if not pyvista.OFF_SCREEN:
        plotter.show()
    else:
        plotter.screenshot("glyphs.png")
    




We now solve the Stokes problem, but using monolithic matrix with the
velocity and pressure degrees of freedom interleaved, i.e. without any
u/p block structure in the assembled matrix. A direct (LU) solver is
used.

In [6]:
# Define variational problem
(u, p) = ufl.TrialFunctions(W)
(v, q) = ufl.TestFunctions(W)
f = Function(Q)
a = form((inner(grad(u), grad(v)) - inner(p, div(v)) - inner(div(u), q)) * dx)
L = form(inner(f, v) * dx)

# Assemble LHS matrix and RHS vector
A = fem.petsc.assemble_matrix(a, bcs=bcs)
A.assemble()
b = fem.petsc.assemble_vector(L)

fem.petsc.apply_lifting(b, [a], bcs=[bcs])
b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)

# Set Dirichlet boundary condition values in the RHS
for bc in bcs:
    bc.set(b)

# Create and configure solver
ksp = PETSc.KSP().create(msh.comm)
ksp.setOperators(A)
ksp.setType("preonly")

# Configure MUMPS to handle pressure nullspace
pc = ksp.getPC()
pc.setType("lu")
pc.setFactorSolverType("superlu_dist")

# Compute the solution
U = Function(W)
try:
    ksp.solve(b, U.x.petsc_vec)
except PETSc.Error as e:
    if e.ierr == 92:
        print("The required PETSc solver/preconditioner is not available. Exiting.")
        print(e)
        exit(0)
    else:
        raise e

# Split the mixed solution and collapse
u, p = U.sub(0).collapse(), U.sub(1).collapse()

plot(u, p)

2025-11-09 11:51:23.065 (   1.360s) [    FFFFBF655720]vtkXOpenGLRenderWindow.:1458  WARN| bad X server connection. DISPLAY=


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…